# Building a Graph Convolutional Network Using Sparse Matrices

This tutorial illustrates step-by-step how to write and train a Graph Convolutional Network ([Kipf et al. (2017)](https://arxiv.org/abs/1609.02907)) using DGL's sparse matrix APIs.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dmlc/dgl/blob/master/notebooks/sparse/gcn.ipynb) [![GitHub](https://img.shields.io/badge/-View%20on%20GitHub-181717?logo=github&logoColor=ffffff)](https://github.com/dmlc/dgl/blob/master/notebooks/sparse/gcn.ipynb)

In [1]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
os.environ['DGLBACKEND'] = "pytorch"

# TODO(Steve): change to stable version.
# Uncomment below to install required packages.
#!pip install --pre dgl -f https://data.dgl.ai/wheels-test/repo.html > /dev/null

try:
    import dgl
    installed = True
except ImportError:
    installed = False
print("DGL installed!" if installed else "Failed to install DGL!")

DGL installed!


## Graph Convolutional Layer

Mathematically, the graph convolutional layer is defined as:
$$f(X^{(l)}, A) = \sigma(\hat{D}^{-\frac{1}{2}}\hat{A}\hat{D}^{-\frac{1}{2}}X^{(l)}W^{(l)})$$
with $\hat{A} = A + I$, where $A$ denotes the adjacency matrix and $I$ denotes the identity matrix, $\hat{D}$ refers to the diagonal node degree matrix of $\hat{A}$ and $W^{(l)}$ denotes a trainable weight matrix. $\sigma$ refers to a non-linear activation (e.g. relu).

The code below shows how to implement it using the `dgl.sparse` package. The core operations are:
* `dgl.sparse.identity` creates the identity matrix $I$.
* The augmented adjacency matrix $\hat{A}$ is then computed by adding the identity matrix to the adjacency matrix $A$.
* `A_hat.sum(0)` aggregates the augmented adjacency matrix $\hat{A}$ along the first dimension which gives the degree vector of the augmented graph.
* `dgl.sparse.diag` creates the diagonal degree matrix $\hat{D}$ from the degree vector.
* `D_hat @ A_hat @_hat` computes the convolution matrix which is then multiplied by the linearly transformed node features.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import dgl.sparse as dglsp

class GCNLayer(nn.Module):
    def __init__(self, in_size, out_size):
        super(GCNLayer, self).__init__()
        self.W = nn.Linear(in_size, out_size)

    def forward(self, A, X):
        ########################################################################
        # (HIGHLIGHT) Compute the symmetrically normalized adjacency matrix with
        # Sparse Matrix API
        ########################################################################
        A_hat = A + dglsp.identity(A.shape)
        D_hat = dglsp.diag(A_hat.sum(0)) ** -0.5
        return D_hat @ A_hat @ D_hat @ self.W(X)

A Graph Convolutional Network is then defined by stacking this layer.

In [3]:
# Create a GCN with the GCN layer.
class GCN(nn.Module):
    def __init__(self, in_size, out_size, hidden_size):
        super(GCN, self).__init__()
        self.conv1 = GCNLayer(in_size, hidden_size)
        self.conv2 = GCNLayer(hidden_size, out_size)

    def forward(self, A, X):
        X = self.conv1(A, X)
        X = F.relu(X)
        return self.conv2(A, X)

## Training the GCN

We then train the GCN model on the Cora dataset for node classification. Note that since the model expects an adjacency matrix as the first argument, we first construct the adjacency matrix from the graph using the `dgl.sparse.from_coo` API which returns a DGL `SparseMatrix` object.

In [4]:
def evaluate(g, pred):
    label = g.ndata["label"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]

    # Compute accuracy on validation/test set.
    val_acc = (pred[val_mask] == label[val_mask]).float().mean()
    test_acc = (pred[test_mask] == label[test_mask]).float().mean()
    return val_acc, test_acc

def train(model, g):
    features = g.ndata["feat"]
    label = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=5e-4)
    loss_fcn = nn.CrossEntropyLoss()

    # Preprocess to get the adjacency matrix of the graph.
    src, dst = g.edges()
    N = g.num_nodes()
    A = dglsp.from_coo(dst, src, shape=(N, N))

    for epoch in range(100):
        model.train()

        # Forward.
        logits = model(A, features)

        # Compute loss with nodes in the training set.
        loss = loss_fcn(logits[train_mask], label[train_mask])

        # Backward.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Compute prediction.
        pred = logits.argmax(dim=1)

        # Evaluate the prediction.
        val_acc, test_acc = evaluate(g, pred)
        if epoch % 5 == 0:
            print(
                f"In epoch {epoch}, loss: {loss:.3f}, val acc: {val_acc:.3f}"
                f", test acc: {test_acc:.3f}"
            )


# Load graph from the existing dataset.
dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

# Create model.
feature = g.ndata['feat']
in_size = feature.shape[1]
out_size = dataset.num_classes
gcn_model = GCN(in_size, out_size, 16)

# Kick off training.
train(gcn_model, g)

Extracting file to /root/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
In epoch 0, loss: 1.957, val acc: 0.122, test acc: 0.130
In epoch 5, loss: 1.932, val acc: 0.200, test acc: 0.210
In epoch 10, loss: 1.897, val acc: 0.386, test acc: 0.433
In epoch 15, loss: 1.851, val acc: 0.518, test acc: 0.571
In epoch 20, loss: 1.788, val acc: 0.542, test acc: 0.569
In epoch 25, loss: 1.706, val acc: 0.710, test acc: 0.729
In epoch 30, loss: 1.606, val acc: 0.746, test acc: 0.780
In epoch 35, loss: 1.491, val acc: 0.756, test acc: 0.787
In epoch 40, loss: 1.366, val acc: 0.770, test acc: 0.789
In epoch 45, loss: 1.237, val acc: 0.768, test acc: 0.789
In epoch 50, loss: 1.111, val acc: 0.772, test acc: 0.795
In epoch 55, loss: 0.995, val acc: 0.770, test acc: 0.796
In epoch 60, loss: 0.891, val acc: 0.772, test

Check out the full example script [here](https://github.com/dmlc/dgl/blob/master/examples/sparse/gcn.py).